In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [13]:
data = pd.read_csv("df_imputed.csv")
data.drop(columns=['Unnamed: 0'], inplace=True)
data

,id,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,...,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU)
0,20001305.0,15.0,23.0,47.0,11.4,108.0,0.8,154.0,142.0,4.3,...,0,0,0,0,1,0,0,0,0,0
1,20001305.0,13.0,25.0,48.0,10.8,107.0,0.9,149.0,140.0,4.7,...,0,0,0,0,1,0,0,0,0,0
2,20001305.0,13.0,24.0,50.0,10.8,108.0,0.9,131.0,141.0,4.1,...,0,0,0,0,1,0,0,0,0,0
3,20001361.0,14.0,22.0,28.0,6.3,107.0,2.5,161.0,137.0,5.8,...,0,0,0,0,1,0,0,0,0,0
4,20001361.0,15.0,20.0,32.0,6.5,108.0,2.5,124.0,137.0,5.7,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93599,29999498.0,20.0,22.0,13.0,11.2,100.0,1.2,76.0,137.0,4.6,...,0,0,0,0,1,0,0,0,0,0
93600,29999498.0,15.0,21.0,13.0,10.1,103.0,1.2,102.0,135.0,4.2,...,0,0,0,0,1,0,0,0,0,0
93601,29999625.0,17.0,21.0,14.0,8.5,104.0,1.2,109.0,142.0,4.6,...,0,0,0,0,0,0,0,1,0,0
93602,29999625.0,15.0,24.0,21.0,8.6,110.0,1.6,122.0,149.0,4.1,...,0,0,0,0,0,0,0,1,0,0


In [14]:
deceased_df = data[data['icu_death'] == 1]
survived_df = data[data['icu_death'] == 0]

deceased_df.head()

,id,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,...,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU)
0,20001305.0,15.0,23.0,47.0,11.4,108.0,0.8,154.0,142.0,4.3,...,0,0,0,0,1,0,0,0,0,0
1,20001305.0,13.0,25.0,48.0,10.8,107.0,0.9,149.0,140.0,4.7,...,0,0,0,0,1,0,0,0,0,0
2,20001305.0,13.0,24.0,50.0,10.8,108.0,0.9,131.0,141.0,4.1,...,0,0,0,0,1,0,0,0,0,0
141,20020590.0,13.0,26.0,38.0,8.6,97.0,1.1,81.0,131.0,4.8,...,1,0,0,0,0,0,0,0,0,1
142,20020590.0,14.0,23.0,40.0,8.4,98.0,1.3,61.0,130.0,4.8,...,1,0,0,0,0,0,0,0,0,1


In [15]:

deceased_df.drop('id', axis=1, inplace=True)
correlation_matrix = deceased_df.corr()
abs_target_correlation = correlation_matrix['los_icu'].abs()

print(abs_target_correlation.drop('los_icu', axis=0))
N = 30 
top_features = abs_target_correlation.drop('los_icu', axis=0).nlargest(N).index.tolist()

X = deceased_df[top_features]
y=deceased_df['los_icu']


D:\anaconda3\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


aniongap                                                          0.113425
bicarbonate                                                       0.072689
bun                                                               0.034244
calcium                                                           0.035887
chloride                                                          0.003144
                                                                    ...   
first_careunit_Neuro Intermediate                                 0.006795
first_careunit_Neuro Stepdown                                     0.007410
first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU)    0.022016
first_careunit_Surgical Intensive Care Unit (SICU)                0.062911
first_careunit_Trauma SICU (TSICU)                                0.009650
Name: los_icu, Length: 97, dtype: float64


In [16]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [17]:
model = RandomForestRegressor(n_estimators=100)

model.fit(X_train, y_train)

loss = model.score(X_test, y_test)  
print(f'Test Score: {loss}') 
predictions = model.predict(X_test)


Test Score: 0.8598806477599479


In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'Test MSE: {mse}')
print(f'Test MAE: {mae}')
print(f"R-squared (R2): {r2}")

Test MSE: 10.755776085512306
Test MAE: 1.7853060365369338
R-squared (R2): 0.8598806477599479


In [19]:
import pandas as pd
predictions_df = pd.DataFrame(predictions, columns=['Predicted LOS_ICU'])
y_test_df = pd.DataFrame(y_test, columns=['los_icu'])
predictions_df.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)
comparison_df = pd.concat([y_test_df, predictions_df], axis=1)
print(comparison_df.head())



   los_icu  Predicted LOS_ICU
0     4.94             4.3112
1     1.01             3.9916
2     2.70             3.9853
3     3.18             8.7373
4     8.96             9.1388


In [20]:
survived_df.drop('id', axis=1, inplace=True)
correlation_matrix = survived_df.corr()
abs_target_correlation = correlation_matrix['los_icu'].abs()

print(abs_target_correlation.drop('los_icu', axis=0))
N = 30  
top_features = abs_target_correlation.drop('los_icu', axis=0).nlargest(N).index.tolist()

X = survived_df[top_features]
y=survived_df['los_icu']


D:\anaconda3\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


aniongap                                                          0.059016
bicarbonate                                                       0.070384
bun                                                               0.034100
calcium                                                           0.038278
chloride                                                          0.014794
                                                                    ...   
first_careunit_Neuro Intermediate                                 0.004587
first_careunit_Neuro Stepdown                                     0.009522
first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU)    0.030888
first_careunit_Surgical Intensive Care Unit (SICU)                0.047393
first_careunit_Trauma SICU (TSICU)                                0.037423
Name: los_icu, Length: 97, dtype: float64


In [21]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)
loss = model.score(X_test, y_test)  
print(f'Test Score: {loss}') 

predictions = model.predict(X_test)

Test Score: 0.6837107311088054


In [22]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'Test MSE: {mse}')
print(f'Test MAE: {mae}')
print(f"R-squared (R2): {r2}")

Test MSE: 12.133077583794742
Test MAE: 1.8575009442113057
R-squared (R2): 0.6837107311088054
